In [8]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "vscode"            # good for VS Code
pio.renderers.default = "notebook_connected"


In [9]:
# because the notebook lives in /notebooks, go up one level to the repo root
REPO_ROOT = Path.cwd().parent
DATA_PATH = REPO_ROOT / "data" / "interview_task_dataset.csv"

# print("CWD:", Path.cwd())
# print("DATA_PATH:", DATA_PATH.resolve())
assert DATA_PATH.exists(), f"File not found: {DATA_PATH.resolve()}"

In [10]:
df = pd.read_csv(DATA_PATH)
df.head(3)

,Record ID,Department,Time Narrative,Worked Time,Charged to Client?,Grade,Category
0,p-0001,a,Amending and updating statement,0.4,YES,Senior,NaN
1,p-0002,a,Reviewed court order and drafted advice email ...,1.3,YES,Junior,NaN
2,p-0003,a,considering email in from counsel attaching FD...,0.3,YES,Junior,"analyse, review, research"


# Step — Load + Inspect, then Label Sparsity

In [19]:
df.shape                    # (rows, cols)


(2157, 7)

In [17]:
df.info()

labelled = df['Category'].notna().sum()
unlabelled = df['Category'].isna().sum()
share = labelled / len(df) * 100

print(f"\nLabelled:   {labelled:,}")
print(f"Unlabelled: {unlabelled:,}")
print(f"Share labelled: {share:.1f}%")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2157 entries, 0 to 2156
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Record ID           2157 non-null   object 
 1   Department          2157 non-null   object 
 2   Time Narrative      2157 non-null   object 
 3   Worked Time         2157 non-null   float64
 4   Charged to Client?  2157 non-null   object 
 5   Grade               2157 non-null   object 
 6   Category            561 non-null    object 
dtypes: float64(1), object(6)
memory usage: 118.1+ KB

Labelled:   561
Unlabelled: 1,596
Share labelled: 26.0%


# Label distribution (only on labelled rows)

What you just did (and why)
What: You grouped by Grade and computed how many entries were charged and the % charged.

Why (business): This shows baseline billing efficiency by seniority. In your data, Seniors have the highest charge rate (~87%), Juniors lower (~77%). This becomes a talking point and feeds the revenue-uplift calc later (e.g., “if Juniors’ charged% rose by 5 pts → £X more/week”).

Why (modelling): Confirms “Charged to Client?” and Grade are informative features and correlate with category. It also flags a risk to watch: if a category is basically “Admin time”, we’ll likely see lower charge%—good to validate but avoid any leakage assumptions.

In [11]:
tbl = (
    df.assign(charged=df["Charged to Client?"].str.upper().eq("YES"))
      .groupby("Grade", as_index=False)
      .agg(rows=("charged", "size"), charged_yes=("charged", "sum"))
)
tbl["charged_%"] = (tbl["charged_yes"] / tbl["rows"] * 100).round(1)
tbl.sort_values("Grade", inplace=True)

tbl

,Grade,rows,charged_yes,charged_%
0,Junior,893,686,76.8
1,Partner,268,224,83.6
2,Senior,996,868,87.1


In [13]:
# sort for a tidy left→right story
plot_df = tbl.sort_values("charged_%", ascending=False).copy()

fig = px.bar(
    plot_df,
    x="Grade", y="charged_%", text="charged_%",
)

fig.update_traces(
    texttemplate="%{text:.1f}%", textposition="outside",
    hovertemplate="Grade: %{x}<br>Charge rate: %{y:.1f}%<br>Rows: %{customdata}",
    customdata=plot_df["rows"],
)

fig.update_layout(
    title=dict(text="Charge rate by grade", x=0.02, xanchor="left"),
    template="simple_white",
    font=dict(family="Segoe UI, Arial, sans-serif", size=14),
    yaxis=dict(title="Charged (%)", range=[0, 100], ticksuffix="%"),
    xaxis_title=None,
    uniformtext_minsize=12,
    bargap=0.35,
    margin=dict(t=60, r=30, l=60, b=60),
)

fig.show()


# Step — Labelled category mix (class balance)

Step — Labelled category mix (class balance)
Purpose (why):

To see how many labelled examples we have per category and whether the classes are balanced or skewed.

Class balance affects model choice, train/validation split, metrics, and may require class weights. Getting this early avoids misleading accuracy.

What we did:

Took only the labelled rows (where Category is not null).

Counted rows per Category and computed each category’s % share of the labelled subset.

Plotted an interactive bar chart so we can immediately spot majority/minority classes.

How (logic):

df_l = df[df["Category"].notna()] → keep labelled only.

value_counts() → get counts by category.

share_% = rows / total * 100 → compute percentage share.

Plot with Plotly to see the mix clearly.

What this tells us:

If one or two categories dominate, a plain accuracy metric will be inflated and misleading.

Minority classes will need help (e.g., class weights during training, stratified splitting, and macro F1 for fair evaluation).

It also hints at where the model may struggle and which categories might need better features or later relabelling.

How we’ll use it next:

We’ll do a stratified train/validation split to preserve these proportions.

Track macro F1 (and per-class recall) rather than only accuracy.

Enable class_weight='balanced' for linear models and tune scale_pos_weight (or equivalent) for tree boosters.

Keep this chart/table for the slides to show you’re risk-aware about imbalance.

In [15]:
df_l = df[df["Category"].notna()].copy()

cat_counts = (
    df_l["Category"]
    .value_counts()
    .rename_axis("Category")
    .reset_index(name="rows")
)
cat_counts["share_%"] = (cat_counts["rows"] / cat_counts["rows"].sum() * 100).round(1)
cat_counts



,Category,rows,share_%
0,client time,199,35.5
1,preparing documents,109,19.4
2,"analyse, review, research",85,15.2
3,Other comms,75,13.4
4,onboarding,49,8.7
5,admin,32,5.7
6,billing,12,2.1


In [ ]:
fig = px.bar(
    cat_counts, x="Category", y="rows", text="share_%",
    title="Labelled subset: category mix",
)
fig.update_traces(texttemplate="%{text}%", textposition="outside")
fig.update_layout(template="simple_white", xaxis_title=None, yaxis_title="Rows")
fig.show()

# Step — Text length (word count) distribution

Why: catches empties/odd entries, guides cleaning and n-gram settings, and helps us decide whether to cap length or treat outliers.

In [20]:
# word count per narrative
df["n_words"] = df["Time Narrative"].fillna("").str.split().str.len()
# quick sanity stats
df["n_words"].describe()

count    2157.000000
mean        8.121465
std         5.025546
min         1.000000
25%         5.000000
50%         7.000000
75%        10.000000
max        54.000000
Name: n_words, dtype: float64

In [21]:
fig = px.histogram(
    df, x="n_words", nbins=40,
    title="Time Narrative length (word count)",
)
fig.update_layout(
    template="simple_white",
    xaxis_title="Words per entry",
    yaxis_title="Rows",
    margin=dict(t=60, r=30, l=60, b=60),
)
fig.show()

What we learned
Narratives are short. Median ≈ 7 words (25% = 5, 75% = 10), long tail up to 54.

Implication: Most entries are brief phrases, not paragraphs. The model won’t get long context from text alone.

Why we did this (value)
Feature strategy: Short texts → TF-IDF with n-grams (1–2 or 1–3) will work well. We don’t need heavy transformers for a POC.

Cleaning decisions: We can keep punctuation/lemmatisation simple; focus on domain tokens and abbreviations (LOE, RA, TOBs, O/S).

Model design: Because text is short, non-text features (Grade, Charged?, Worked Time) will add real signal—so we’ll include them in the model.

Quality flags: Very short entries (e.g., ≤2–3 words) are low-info; we can tag them so:

the classifier leans more on non-text features, and

the app shows lower confidence for these cases.

Robustness: A few very long entries (20–50+ words) are outliers. We can clip at a safe cap (e.g., 40 words) without losing meaningful info.

# One tiny next step — mark “low-info” narratives

Why this helps:

We’ll treat these rows cautiously in training and in the app (e.g., display a confidence warning).

It justifies combining text with Grade / Charged? / Worked Time features.

In [22]:
# Flag low-information narratives (≤ 3 words) so we know where text is weak
df["low_info"] = (df["n_words"] <= 3).astype(int)

# Quick view: how common are they?
low_info_share = (df["low_info"].mean() * 100).round(1)
print(f"Low-info narratives (≤3 words): {low_info_share}%")

Low-info narratives (≤3 words): 14.9%


In [23]:
# Optional: see if low-info rows are less likely to be labelled (useful hint for modelling difficulty)
lab_rate = (
    df.assign(labelled=df["Category"].notna())
      .groupby("low_info")["labelled"].mean()
      .mul(100).round(1)
      .rename("labelled_%")
)
lab_rate

low_info
0    26.4
1    23.9
Name: labelled_%, dtype: float64

What this “low-info narratives” step tells us (and why it matters)
What we found: ~14.9% of narratives have ≤3 words. Those rows are harder to label (23.9% labelled) than normal rows (26.4% labelled).

Why we did it: very short text has weak signal. Knowing its share up front shapes our feature strategy, evaluation, and app UX.

Immediate, practical uses
Feature strategy: keep a binary feature low_info in the model. Use TF-IDF with n-grams (1–2/1–3) so short phrases like “court hearing” still have power.

Use non-text features more: for low-info rows, Grade / Charged? / Worked Time will carry extra signal—good to include them.

Evaluation: report metrics by low_info segment (so we know where the model struggles), and prefer macro F1 over plain accuracy.

App UX: if low_info==1 (or prob < threshold), show a “low confidence” hint to the user.

Production monitoring: track % low_info as a data-quality KPI. If it spikes (say > +5 ppt from baseline 15%), alert & consider re-training.

# Next tiny step — Worked Time vs Charged? (one cell)


In [25]:
dfv = df.assign(charged=df["Charged to Client?"].str.upper().map({"YES": "Yes", "NO": "No"}))

fig = px.box(
    dfv, x="charged", y="Worked Time", color="charged",
    title="Worked Time vs Charged?",
    points="suspectedoutliers",
)
fig.update_layout(template="simple_white", xaxis_title=None, yaxis_title="Hours")
fig.show()

In [26]:
dfv.groupby("charged")["Worked Time"].agg(count="size", median="median", mean="mean").round(2)

,count,median,mean
charged,,,
No,379,0.5,0.66
Yes,1778,0.3,0.58


Why we did it
To compare the time distribution of entries that were charged to client vs not charged. This tells us if “Worked Time” is a useful signal and where money leaks might be.

What it shows (your numbers)
Counts: Yes = 1,778, No = 379 → most entries are charged.

Medians: Yes 0.3h, No 0.5h → non-charged entries are slightly longer on average.

Means: Yes 0.58h, No 0.66h.

Tail: Charged has big outliers (6–12h court days etc.). Non-charged rarely that long.

Why this is useful
Feature design: Keep “Worked Time” as a feature. Consider bucketing (≤0.25, 0.25–0.75, 0.75–2, >2h) and/or a log transform or winsorising the top 1–5% to reduce outlier impact.

Interactions: Time × Grade can help (e.g., long Partner hours vs Admin).

Business talking point: There are long non-billable blocks (median 0.5h) → potential recovery/efficiency opportunity. Also shows where revenue lives (very long charged sessions).

Monitoring later: Track avg non-billable worked time as a KPI.

Is the visual useful?
Yes. For non-technical stakeholders it clearly shows:

non-billable entries aren’t just tiny scraps; they can be chunky,

the fat right tail of charged work = big revenue sessions.